# Biomass accumulation {#sec-biomass}

## Introduction

Previously, we modeled the growth of a whole-plant using a logistic function characterized by an early exponential growth phase, a linear phase, and a later saturating phase to reach the final biomass. Here, we will modify and expand the model to represent some basic physiological processes. That is, we are going to make our model a little more mechanistic. In the logistic growth model, we assumed that the growth rate slows down as the plant reaches its potential biomass (this is analogous to running out of space or resources in a density dependent population). There could be other reasons or mechanisms for plants to slow down their growth as they mature. One possible mechanism is that the efficiency of growth may decline over time as the plant ages. For example, the efficiency of resource acquisition and use (e.g. light) per carbon invested in foliage tissue will slow down as plants grow bigger with increasingly overlapping foliage that results in self-shading. In addition, the efficacy of a leaf, even if it's not shaded, for producing photosynthates would also decline over time as the leaf ages. At the lower level of organizations, a constant turn over of tissues, cells, proteins and metabolites takes place. That is, old tissue, cells, or molecules die or degrade while new tissue, cells, or molecules are synthesized. Here we will explore some of the simple but physiologically meaningful ways to model realistic plant growth patterns. As done in the previous examples, we will be simulating biomass accumulation of a whole-plant from seed to maturity for an annual plant.

## Why balance carbon?

Before we move further ahead, let's take a look at some basic physiology of how an organism could accumulate (gain) or dissipate (lose) its biomass through gas-exchange. This would apply to both animals and plants. We just need to remember the law of conservation of mass, remind ourselves that gases have their own mass, and practice the skill to balance input and output of mass through stoichiometry of chemical elements like managing your bank account. In plants, we do this mass balancing based on carbon as its currency because carbon is one of the most abundant chemical elements forming plant biomass, and its concentration (mass/mass) is consistent within organ types (e.g., leaf, stem, root, fruit, seeds) and conserved within functional groups and taxa. Across different taxa, carbon concentration in plants stays between 40% and 50% with 45 % being a reasonable approximation for most parts and taxa [@Ma2018; @Taiz2018]. Herbaceous plants tend to have lower carbon content ($\approx 42$ %) while trees, and conifers in particular, tend to have higher carbon content ($\approx 50$ %) [@Ma2018]. The majority of carbon goes in and out of plants as gas through photosynthesis (a.k.a., assimilation) and respiration. For this reason modeling gas-exchange processes become fundamental in plant ecophysiological models [@Kim2020]. While somewhat underappreciated presumably, carbon lost through respiration can also account for a substantial amount of biomass (or weight) loss in animals and humans
[@Meerman2014].

::: {#exr-breathing}
### Biomass loss through respiration
Do humans and animals lose mass through breathing? How much [See @Meerman2014]? Guesstimate potential mass change through breathing for 10 days for a person whose body mass is 70 kg assuming a constant breathing (e.g., gas-exchange) rate of 6.0 L min^−1^ (0.5 L per breathing, 12 breaths/minute) and respiratory quotient (RQ = Volume of released/Volume of absorbed) of 0.8 [@Meerman2014; @Patel2023]. Humans do not absorb all of the inhaled O_2 for metabolism; inhaled air contains 21% of O_2 while exhaled breath contains approximately 16% by volume [@Pleil2021]. Provide both absolute and relative mass changes for this period through breathing only.
:::

In [ ]:
using Cropbox
@system breath(Controller) begin
    t(context.clock.time) ~ track(u"d")
    W0: initial_body_mass      => 70   ~ preserve(parameter, u"kg")
    RQ: respiratory_quotient   =>  1.0 ~ preserve(parameter)
    br: breathing_rate         =>  6.0 ~ preserve(parameter, u"l/minute")
    CO2: MW_of_CO2             => 44.0 ~ preserve(parameter, u"g")
    O2: MW_of_O2               => 32.0 ~ preserve(parameter,u"g")
    V: molar_air_volume        => 22.4 ~ preserve(parameter, u"l")
    air_in(O2, br, V): inhale_MW => 0.21*O2*br/V ~ track(u"g/minute")
    air_out(O2, CO2, br, RQ, V): exhale_MW   => (0.16*O2+0.05*RQ*CO2)*br/V ~ track(u"g/minute")
    W(air_in, air_out): body_mass =>  begin
        air_in - air_out
    end ~ accumulate(init = W0, u"kg")
    ΔW(air_in, air_out): delta_mass => begin
        air_in - air_out
    end ~ accumulate(init = 0, u"kg")
    # ΔW_lbs(ΔW): total_mass_change_lbs => begin
    #     ΔW |> u"lb"
    # end ~ track(u"lb")
    # r(W, W0): relative_change => begin
    #     (W - W0)/W0 * 100u"percent"
    # end ~ track(u"percent")     
end

In [ ]:
c0 = :Clock => :step => 1u"hr"
stop = 10u"d"

## Processes governing biomass dynamics

### Photosynthesis and respiration

Let's think about ways to tailor a simple growth function in the form of $\frac{dW}{dt}=rW$ to account for the basic physiological mechanisms that are specific to plants. Three key physiological processes that determine the rate of plant growth are photosynthesis, respiration, and allocation of assimilates. Photosynthesis assimilates CO<sub>2</sub> to provide carbohydrates as the substrate for respiration. Respiration breaks down the photosynthates to supply energy needed for maintenance of existing biomass as well as C skeletons and energy needed for construction of new biomass (growth).

Plants will grow if the rate of photosynthate production outpaces the rate of respiration for the whole-plant. If they are equal, then the plant maintains the status quo. Incorporating this simple relationship between photosynthesis and respiration on the rate of biomass accumulation, the absolute (or actual) growth rate (AGR; $\frac{dW}{dt}$) can be written as:

$$
    \frac{dW}{dt} = rW = c(P_{g}-R_{pl})
$${#eq-AGR}

Where $P_g$ is the production rate of assimilates by the whole-plant per unit time (in g~C~ d^−1^, g d^−1^, or g d^−1^). $R_{pl}$ is the whole-plant respiration rate per unit time in the same unit as the assimilates accounting for both maintenance and growth components. $c$ is an appropriate conversion factor for the unit of substrate considered.

::: {#nte-negative-growth .callout-note}

Can actual plant growth rate @eq-AGR be negative in this formulation? Under what circumstances can that happen? '
What does that mean physiologically? Can that happen in reality? If so, under what circumstances can that happen? Discuss in both physiological and mathematical contexts.
:::

@Thornley1970 expanded @eq-AGR elegantly to show how the above relationship translates to RGR, photosynthesis, and two components of respiration (growth and maintenance) as follows. This is a simple but robust representation of the photosynthesis-respiration relationship commonly applicable to all plant taxa.

$$
r = Y_g \cdot c_1 \frac{P_g - k_m W}{W} = Y_g \cdot c_1 \left( \frac{P_g}{W} - k_m \right) = Y_g \cdot c_1 \left( P_{g_m} - k_m \right)
$${#eq-thornley-model}
  

Where $r$ is relative growth rate (g g d^−−1^), $Y_g$ is the growth yield (units of C appearing in product W per unit of C substrate utilized for growth), $P_g$ is the gross daily production of photosynthates (in ) for the whole-plant (g~CH_2O~ d^−1^), $k_m$ is the coefficient of maintenance respiration (\_CH_2O  \_dm\^-1  \^-1 ) , and $W$ is the whole-plant biomass (g) measured as dry matter (i.e., g~dm~). $c_1$ is the conversion factor from to biomass assuming 45% of biomass is carbon ($c_1$ = (12.0/30.0)/0.45) $\approx$ 0.889).

As shown in @sec-growth with the Gompertz equation, the growth yield ($Y_g$) may decline over time as the plant ages.

:::{#tbl-thornley-growth-parms}
| Symbol | Value | Units | Description |
| :-- | :-- | :-- | :------------ |
| $t$ | - | $\mathrm{d}$ | Time unit used in the model |
| $\alpha$ | 0.03 | $\mathrm{d^{-1}}$ | Decay rate of the growth efficiency over time |
| $Y_{g_0}$ | 0.75 | - | Potential growth yield of the biomass conversion process |
| $Y_g$ | - | - | Actual growth yield of the biomass conversion process |
| $c_1$ | 0.889 | - | Conversion factor from $\mathrm{CH_2O}$ to biomass assuming 45% carbon in biomass |
| $P_g$ | - | $\mathrm{g}$ $\mathrm{d^{-1}}$ | Substrate (as $\mathrm{C}$, $\mathrm{CO_2}$, $\mathrm{CH_2O}$) production rate by a whole-plant per unit time |
| $P_{g_m}$ | 0.4 | $\mathrm{g}$ $\mathrm{g^{-1}}$ $\mathrm{d^{-1}}$ | Substrate (as $\mathrm{CH_2O}$) production rate per whole-plant mass per unit time |
| $k_m$ | 0.015 | $\mathrm{g}$ $\mathrm{g^{-1}}$ $\mathrm{d^{-1}}$ | Coefficient of maintenance respiration |
| $r$ | - | $\mathrm{g}$ $\mathrm{g^{-1}}$ $\mathrm{d^{-1}}$ | Relative growth rate |
| $W_0$ | 0.15 | $\mathrm{g}$ | Seed mass or initial biomass |
| $W$ | - | $\mathrm{g}$ | Whole-plant biomass as dry matter |

Thornley growth model variables and parameters 
:::

:::{#exr-thornley-model}
### Thornley growth model
Implement the growth model by @Thornley1970 into a system in `Cropbox` and examine its behavior using the information provided in @tbl-thornley-growth-parms. Assume the $Y_g$ declines over time at a rate that given by $\alpha$. Mull over if this assumption is valid and if so, what plant or canopy process can represent this decline over time.

* Evaluate the dependency of your model using `dependency()`. 
* List the parameters and their values using `parameters()`.
* Run your simulation on daily step for 300 days.
* Compare the results with those from @sec-growth. 
:::

In [ ]:
#| output: false

@system Thornley1970(Controller) begin
    t(context.clock.time) ~ track(u"d")

    α:         decay_rate             => 0.03  ~ preserve(parameter, u"d^-1")
    Yg0:       potential_growth_yield => 0.75  ~ preserve(parameter)
    Yg(α, Yg): growth_efficiency      => -α*Yg ~ accumulate(init = Yg0)

    c1:   conversion_factor        => 12/30/0.45 ~ preserve(parameter)
    
    β:         production_decay           => 0.0  ~ preserve(parameter, u"d^-1")
    Pg_m0: max_production_rate_per_mass => 0.4        ~ preserve(parameter, u"g/g/d")
    Pg_m(β, Pg_m): production_rate_per_mass => -β*Pg_m ~ accumulate(init=Pg_m0, u"g/g/d")
    km:   maintenance_coeff        => 0.015      ~ preserve(parameter, u"g/g/d")

    r(Yg, c1, Pg_m0, km): intrinsic_RGR => begin
        Yg * c1 * (Pg_m0 - km)
    end ~ track(u"g/g/d")

    W0:      initial_biomass => 0.15 ~ preserve(parameter, u"g")
    W(r, W): biomass         => r*W  ~ accumulate(init = W0, u"g")
end

Cropbox.dependency(Thornley1970)

parameters(Thornley1970)

c = :Clock => :step => 1u"d"
t70 = simulate(Thornley1970, config = c, stop = 300u"d")

plot(t70, :t, [:W0, :W], kind = :line);

### Carbon allocation

Another important process that determines plant growth, biomass accumulation, and eventually crop yield is the partitioning of photosynthates to different plant organs. Allocation or partitioning of the photosynthates is a critical component that affects the rate of plant growth because it determines the destination and fate of these substrates about how and where they are used. For example, some assimilates will be transported to roots to function as substrates for providing ATPs to maintain the ion gradients while others can be invested back to build more new leaves. More investment in photosynthetic machinery (i.e. allocation to foliage) will yield greater return of biomass at a faster rate; this may be analogous to re-investing the earnings to buy more assets in the market.

A simple but mechanistic approach introduced by @Monsi1960 provides useful insights about the intricate relationships between photosynthesis, respiration, and allocation, and their trade-offs with respect to whole-plant growth. The trade-offs of allocating photosynthates between photosynthetic and non-photosynthetic parts can be particularly insightful and relevant for the growth of crops like lettuce and cabbage for which the foliage biomass accounts for the majority of harvested yield or crops with large underground storage parts like radish, potato, garlic, and onions. Carbon allocation to non-photosynthetic parts is also functionally important in herbaceous perennials with storage organs such as rhizomes and woody plants for which non-living tissues (i.e. wood and bark) continue to be part of the whole-plant biomass. [@Monsi1960] intuitively illustrated the significance of the carbon allocation that drives the intricate relationships between carbon gain through assimilation ($A$) and respiration ($R$) based on the fraction of assimilates partitioned to photosynthetic ($m$) and non-photosynthetic tissues ($n$) of a whole-plant. In this approach, the relative growth rate ($r$) can be written as [@Kim2020]:

$$
\begin{split}
    &r= c_2(mA-nR) = c_2(mA-(1-m)R) = c_2(m(A+R)-R)  
\end{split}
$${#eq-monsi-model}

Where $m$ is the fraction of photosynthetic biomass or foliage. This is also known as leaf mass ratio (LMR). Accordingly, $(1-m)$ noted as $n$, represents the fraction of non-photosynthetic biomass (e.g., roots, woody stems). $A$ and $R$ represent the net photosynthetic rate per g of foliage biomass and the respiration rate per g of whole-plant biomass, respectively. Both are in \_  \_dm\^-1  \^-1. $c_2$ is the conversion factor from to biomass assuming 45% of biomass is carbon ($c_2$ = (12.0/44.0)/0.45) $\approx$ 0.606). This approach demonstrates that multiple pathways, not just plants' ability to fix carbon, contribute to maintaining the plant growth rate through the coordination of assimilate production, assimilate allocation to photosynthetic tissues, and assimilate loss through respiration.

Despite considerable advances made in understanding the individual processes of photosynthesis, respiration, sugar metabolism, long-distance transport, and cell growth [@Marcelis1998], the integration of these processes into a biomass-partitioning model is a bottleneck in crop simulation models that operate at the crop level. [@Marcelis1998] provides an outstanding review of modeling approaches for dry-matter partitioning that is still relevant to this date. Briefly, most commonly used method is *descriptive allometry*. It is a primarily empirical approach that partitions photosynthates for biomass production across different plant parts or organs according to predetermined partitioning coefficients. These partitioning coefficients are empirically determined and often vary with crop developmental stage as a function of thermal unit accumulation [@G&vL-1994].

:::{#nte-thornley-monsi .callout-important}
Examine how @eq-thornley-model and @eq-monsi-model are related.

1.  Name the three key processes represented in the models and discuss how each process is represented.
2.  Show mathematically that these two equations are equivalent to represent the same quantity
3.  Discuss the similarities and differences in their approaches.
:::

Both models are simple but robustly grounded in physiology to provide critical insights for understanding the intricate relationship between these three processes and their trade-offs determining plant growth. The trade-offs of allocating photosynthates between photosynthetic and non-photosynthetic parts can be particularly insightful and relevant for the growth of crops like lettuce and cabbage for which the foliage biomass accounts for the majority of harvested yield or crops with large underground storage organs like radish, potato, garlic, and onions. Carbon allocation to non-photosynthetic parts is also functionally important in  herbaceous perennials with storage organs such as rhizomes and woody plants for which dead parts (i.e. wood and bark) continue to be part of the whole-plant biomass.

::: {#exr-monsi-model}
### Monsi growth model
Implement the growth model by Monsi (1960) into a system in Cropbox and examine its behavior using the information provided in @tbl-monsi-parms. Try changing the parameter values and visualize model responses. Assume both $A$ and $R$ declines over time at a rate given by $\alpha$ . Also note that 1) $A$ and $R$ are expressed in $g$ of in hourly time step and 2) photosynthesis requires light.

-   Evaluate the dependency of your model using dependency() function of the Cropbox framework.
-   List the parameters and their values using parameters() function of the Cropbox framework.
-   Run your simulation on daily and hourly steps for 300 days.
-   Compare the results with the Thornley model. If they are different discuss why and what.
:::


:::{#tbl-monsi-parms}
| Symbol | Value | Units | Description |
| :-| :- | :- | :------- |
| $t$ | - | $\mathrm{d}$ | Time unit used in the model |
| $c_2$ | 0.606 | - | Conversion factor from $\mathrm{CO_2}$ to biomass assuming 45% carbon in biomass |
| $m$ | 0.5 | - | Fraction of photosynthetic biomass |
| $n$ | 0.5 | - | Fraction of non-photosynthetic biomass ($1 - m$) |
| $A_{\mathrm{max}}$ | 0.08 | $\mathrm{g}$ $\mathrm{g^{-1}}$ $\mathrm{hr^{-1}}$ | Maximal net $\mathrm{CO_2}$ assimilation rate |
| $R_{\mathrm{max}}$ | 0.006 | $\mathrm{g}$ $\mathrm{g^{-1}}$ $\mathrm{hr^{-1}}$ | Maximal respiration rate |
| $\alpha$ | 0.03 | $\mathrm{d^{-1}}$ | Decay rate of the growth efficiency over time |
| $A$ | - | $\mathrm{g}$ $\mathrm{g^{-1}}$ $\mathrm{d^{-1}}$ | Actual net $\mathrm{CO_2}$ assimilation rate per day during photoperiod |
| $R$ | - | $\mathrm{g}$ $\mathrm{g^{-1}}$ $\mathrm{d^{-1}}$ | Actual respiration rate per day |
| $D_L$ | 12 | $\mathrm{hr}$ | Day length (i.e. photoperiod) |
| $D$ | 24 | $\mathrm{hr}$ | Total hours per day |
| $r$ | - | $\mathrm{g}$ $\mathrm{g^{-1}}$ $\mathrm{d^{-1}}$ | Relative growth rate |
| $W_0$ | 0.15 | $\mathrm{g}$ | Seed mass or initial biomass |
| $W$ | - | $\mathrm{g}$ | Whole-plant biomass as dry matter |

Monsi model variables and parameters
:::

In [ ]:
@system Monsi1960(Controller) begin
    t(context.clock.time) ~ track(u"d")

    c2: conversion_factor => 12/44/0.45 ~ preserve(parameter)    

    m:    leaf_mass_ratio   => 0.5 ~ preserve(parameter)
    n(m): non_leaf_fraction => 1-m ~ track

    Amax: maximal_assimilation => 0.08  ~ preserve(parameter, u"g/g/hr")
    Rmax: maximal_respiration  => 0.005 ~ preserve(parameter, u"g/g/hr")
    α:    decay_rate           => 0.03  ~ preserve(parameter, u"d^-1")

    A(α, A): hourly_assimilation_per_mass => -α*A ~ accumulate(init = Amax, u"g/g/hr")
    R(α, R): hourly_respiration_per_mass  => -α*R ~ accumulate(init = Rmax, u"g/g/hr")

    DL: day_length  => 12 ~ preserve(parameter, u"hr")
    D:  total_hours => 24 ~ preserve(parameter, u"hr")

    A′(A, DL, D): assimilation_per_mass => A * (DL/D) ~ track(u"g/g/d")
    R′(R):        respiration_per_mass                ~ track(u"g/g/d")

    r(c2, m, n, A′, R′): RGR => c2 * (m*A′ - n*R′) ~ track(u"g/g/d")

    W0:      initial_biomass => 0.15 ~ preserve(parameter, u"g")
    W(r, W): biomass         => r*W  ~ accumulate(init = W0, u"g")
end
parameters(Monsi1960)

Cropbox.dependency(Monsi1960)

m60 = simulate(Monsi1960, config = c, stop = 300u"d")

visualize(m60, :t, :W, kind=:line)

p = visualize(t70, :t, :W, kind = :line, name = "Thornley (1970)")
visualize!(p, m60, :t, :W, kind = :line, name = "Monsi (1960)");

### Sink strength

A notable modeling approach for carbon partitioning applies the concept of *sink regulations* [@Marcelis1996]. This approach partitions assimilates among plant organs based on their *sink strength*, which is defined as the competitive ability or potential capacity of an organ for assimilate accumulation [@Marcelis1996]. In this model, the fraction ($f$) of biomass partitioned into $i_{\mathrm{th}}$ sink organ is determined by its sink strength ($S_i$) as:

$$
    f_i = \frac{S_i}{\sum S} \approx \frac{Y_{pot,i}}{\sum Y_{pot}}
$${#eq-sink-strength}

This model illustrates that biomass partitioning (i.e., fractional distribution of biomass) to an organ varies with its competitive sink strength among all sinks which may be be approximated by the potential growth rate of the organ ($Y_{pot,i}$) relative to the total (i.e., sum of all) potential growth rate as shown in @eq-sink-strength. This particular case assumes that all sinks have the same affinity (or get the same priority) for assimilates. The potential growth rate ($Y_{pot,i}$) of sink organ $i$ can be modeled using a sigmoidal growth function such as logistic, Gompertz, Chanter, or Richards function. When the affinity for assimilates differs between organs, @eq-sink-strength can be expanded to account for variable responses to assimilate supply between organs with differing affinity using a function like the Michaelis-Menten equation [@Marcelis1994]. This *sink priority* model by [@Marcelis1994] relies on the simplifying assumption that the source (or transport) path is non-dominant in regulating dry-matter partitioning [@Marcelis1998].

A different approach that accounts for the *transport-resistance* network from source to sink is to simulate the phloem transport of assimilates based on Ernst Munch's hypothesis of bulk flow, in which assimilates are driven by an osmotically generated pressure gradient between source and sink organs and the kinetics of assimilate unloading at the sink [@Minchin1993; @Minchin2005].

### Senescence and decline of biomass

Plants, especially annuals, will lose their biomass. Cells, tissues, and organs will senesce, die, and disintegrate eventually, and are no longer part of the living "biomass". The models we've looked at so far do not account for this aspect of senescence and death. Think about how the processes and mechanisms associated with senescence and death of plant can be implemented in these models. Some useful information may be found in @tbl-senescence.

:::{#tbl-senescence}
| Symbol | Value | Units | Description |
| :- | :- | :- | :------- |
| $\alpha_{\mathrm{sen}}$ | 100 | $\mathrm{d}$ | Plant age (days) at which senescence begins |
| $k_s$ | 0.01 | $\mathrm{g}$ $\mathrm{g^{-1}}$ $\mathrm{d^{-1}}$ | Senescence rate of degradable biomass over time |
| $f_d$ | 1.0 | - | Fraction of degradable biomass |
| ... | - | - | ... |
Potential senescence parameters
:::

Here's an example of how senescence process may be incorporated into the Monsi model.

```julia
@system MyPlantModel(Controller) begin
    t(context.clock.time) ~ track(u"d")

    c2: conversion_factor => 12/44/0.45 ~ preserve(parameter)    

    m:    leaf_mass_ratio   => 0.5 ~ preserve(parameter)
    n(m): non_leaf_fraction => 1-m ~ track

    Amax: maximal_assimilation => 0.08 ~ preserve(parameter, u"g/g/hr")
    Rmax: maximal_respiration  => 0.006 ~ preserve(parameter, u"g/g/hr")

#    α:       decay_rate                   => 0.03 ~ preserve(parameter, u"d^-1")
    α_max: maximum_decay_rate => 0.03    ~ preserve(parameter, u"d^-1")
    k: delay_coeff            => 0.01    ~ preserve(parameter, u"d")    
    α0: initial_decay         => 0.0     ~ preserve(parameter, u"d^-1")
    
    α(k,α_max,α): decay_rate    => k*(α_max - α) ~ accumulate(init = α0 , u"d^-1")

    A(α, A): hourly_assimilation_per_mass => -α*A ~ accumulate(init = Amax, u"g/g/hr")
    R(α, R): hourly_respiration_per_mass  => -α*R ~ accumulate(init = Rmax, u"g/g/hr")

    DL: day_length  => 12 ~ preserve(parameter, u"hr")
    D:  total_hours => 24 ~ preserve(parameter, u"hr")

    A′(A, DL, D): assimilation_per_mass => A * (DL/D) ~ track(u"g/g/d")
    R′(R):        respiration_per_mass                ~ track(u"g/g/d")

    r(c2, m, n, A′, R′): RGR => c2 * (m*A′ - n*R′) ~ track(u"g/g/d")

    s_max: maximum_senescence_rate => 0.02    ~ preserve(parameter, u"d^-1")
    ks: senescence_delay_coeff     => 0.01    ~ preserve(parameter, u"d")    
    s0: initial_senescen           => 0.0     ~ preserve(parameter, u"d^-1")
    
    s(ks,s_max,s): senescence_rate    => ks*(s_max - s) ~ accumulate(init = s0 , u"d^-1")

    W0:          initial_biomass => 0.15       ~ preserve(parameter, u"g")
    W(r, s, W): biomass         => (r - s)*W ~ accumulate(init = W0, u"g")
end
mp = simulate(MyPlantModel, config = :Clock => :step => 1u"d", stop = 300u"d")
visualize(mp, :t, :W, kind = :line)
visualize(mp, :t, [:A′, :R′], kind = :line)
visualize(mp, :t, :r, kind = :line)
visualize(mp, :t, :s, kind = :line)
visualize(mp, :t, :(r - s), kind = :line, ylim = (-0.1, 0.3))
```
You may construct the same system with much less code by reusing `Monsi1960` system as a mix-in.

```julia
@system MyPlantModel2(Monsi1960, Controller) begin
    ks:          senescence_rate => 0.01       ~ preserve(parameter, u"g/g/d")
    W(r, ks, W): biomass         => (r - ks)*W ~ accumulate(init = W0, u"g")
end
```

## Homework Problems
You will expand the models you worked with so far to build a model to satisfy the following conditions: 

Assuming that you are growing plants in a controlled growth chamber with the environmental conditions described in the table, build a simple plant growth model for a plant of your choice for which the entire above-ground biomass will be harvested at the end of one growing season like in a bioenergy crop; that is, no specific organ represents its yield but the entire shoot biomass does. Use the parameters and their values listed in @tbl-thornley-growth-parms and @tbl-monsi-parms as appropriate for your model. Answer the following questions using your model or based on your model.

1. Decide a plant species for which you will develop the growth model. Why did you choose this species?
2. Implement model using Cropbox and plot the biomass trajectory over time using the parameter values provided in Table 5.1 as the initial parameter values. When is the maximum biomass reached and what is that biomass? Do the values look reasonable for the species of your choice based on your knowledge about the plant? 
3. Imagine that you are a plant breeder trying to increase the biomass that can be harvested at the end of the season for this energy crop. What breeding strategies (e.g., increase photosynthesis, increase leaf area) would you use? Can you test your ideas using the model you just developed? How? If not, why not?
4. Using your model, let's try all means available to increase the biomass at harvest. How much more biomass did you gain compared to the initial model prediction? How did you achieve that? Do you think you could have achieved the same outcome in other ways? If so, how or why not? 
5. Describe whether or not and how your model represents the properties and behavior of the system you are trying to simulate satisfactorily. What is your model good for and what are its shortcomings and limitations? 